In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/finaldataset/finals.csv
/kaggle/input/hallucination-dataset/hallucination_dataset.csv


In [4]:
# Cell: Clean Setup and Install (run this first, then RESTART KERNEL)
import os
import shutil

# Clear any existing cache to avoid corruption
cache_dir = '/root/.cache/huggingface'  # Default HF cache in Kaggle
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print("Cleared Hugging Face cache")

working_cache = '/kaggle/working/cache'
if os.path.exists(working_cache):
    shutil.rmtree(working_cache)
    print("Cleared working cache")
os.makedirs(working_cache, exist_ok=True)

# # Install a SINGLE stable version (no conflicts)
# !pip uninstall -y transformers tokenizers  # Remove any broken installs first
# !pip install transformers==4.41.2 datasets torch scikit-learn pandas numpy

# print("Installation complete. NOW RESTART YOUR KERNEL (in Kaggle: Run > Restart Session) and continue from imports.")

Cleared working cache


In [5]:
!pip install peft  # Add this if not installed

In [6]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer

2025-09-14 09:36:22.052409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757842582.263514      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757842582.328977      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Add this cell before dataset creation (cell 25)
!rm -rf train_hallucination val_hallucination test_hallucination

In [7]:
from transformers import Trainer, TrainingArguments

In [8]:
device='cuda'if torch.cuda.is_available() else 'cpu'

In [9]:
print(device)

cuda


In [10]:
import torch
import torch.nn as nn
# from transformers import AutoModel, Trainer, TrainingArguments
from datasets import load_from_disk

In [12]:
# Verify version
import transformers
print(f"Transformers version: {transformers.__version__}")

Transformers version: 4.52.4


In [13]:
import torch
import pandas as pd

In [14]:
df = pd.read_csv('/kaggle/input/finaldataset/finals.csv')

In [15]:
df.columns

Index(['prompt', 'response', 'agent_type', 'Conciseness Score',
       'Instruction-Following Score', 'Persona Adherence Score',
       'Assumption Control Score', 'user_frustration_proxy', 'overconfidence',
       'semantic_alignment', 'semantic_drift', 'repetition_percentage',
       'contradiction_detected', 'assumption_risk', 'sentence_variety_score',
       'keyword_recall', 'extraneous_ratio', 'alignment_balance',
       'spelling_errors', 'number_mismatch_count'],
      dtype='object')

In [16]:
df.isnull().sum()

prompt                         0
response                       0
agent_type                     0
Conciseness Score              0
Instruction-Following Score    0
Persona Adherence Score        0
Assumption Control Score       0
user_frustration_proxy         0
overconfidence                 0
semantic_alignment             0
semantic_drift                 0
repetition_percentage          0
contradiction_detected         0
assumption_risk                0
sentence_variety_score         0
keyword_recall                 0
extraneous_ratio               0
alignment_balance              0
spelling_errors                0
number_mismatch_count          0
dtype: int64

In [16]:
len(df)

35

In [17]:
# df=df[:15]

## Model pre processing

In [17]:
def bert_input_string(idx: int, data: pd.DataFrame) -> str:
    """
    Generates a BERT prompt string by fetching all required features from a
    single row of a pandas DataFrame.

    Args:
        idx (int): The index of the row to fetch data from.
        data (pd.DataFrame): The DataFrame containing the input features.

    Returns:
        str: The formatted prompt string ready for a BERT model.
    """
    # Fetch the row at the specified index
    row = data.iloc[idx]

    # Extract all the features from the row
# Extract all the features from the row
    prompt = row['prompt']
    response = row['response']
    user_frustration_proxy = row['user_frustration_proxy']
    overconfidence = row['overconfidence']
    semantic_alignment = row['semantic_alignment']
    semantic_drift = row['semantic_drift'] # Not used in prompt
    repetition_percentage = row['repetition_percentage']
    contradiction_detected = row['contradiction_detected']
    assumption_risk = row['assumption_risk']
    sentence_variety_score = row['sentence_variety_score'] # Not used in prompt
    keyword_recall = row['keyword_recall']
    extraneous_ratio = row['extraneous_ratio'] # Not used in prompt
    spelling_errors = row['spelling_errors']
    
    # --- Refined Prompt ---
    input_bert = f"""
    [CONTEXT]
    Analyze the following user-AI interaction based on the provided text and pre-computed quality metrics.
    
    [USER PROMPT]
    {prompt}
    
    [AI RESPONSE]
    {response}
    
    [QUALITY METRICS]
    - User Frustration Proxy: {user_frustration_proxy:.2%}
    - Overconfidence Score: {overconfidence:.2%}
    - Semantic Alignment Score: {semantic_alignment:.2%}
    - Repetition Percentage: {repetition_percentage:.2%}
    - Assumption Risk Score: {assumption_risk:.2%}
    - Keyword Recall: {keyword_recall:.2%}
    - Spelling Error Count: {spelling_errors}

    [TASK]
    Based on all the information above, predict the following score:
    - Instruction-Following Score
    """ 
    
    return input_bert
    

In [18]:
df['input_string'] = [bert_input_string(i, df) for i in range(len(df))]
df['hallucination_score'] = df['Instruction-Following Score'].astype(float)

In [19]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

train_val, test = train_test_split(df, test_size=0.1, random_state=42)
train, val = train_test_split(train_val, test_size=0.1111, random_state=42)
train = train.rename(columns={'hallucination_score': 'labels'})
val = val.rename(columns={'hallucination_score': 'labels'})
test = test.rename(columns={'hallucination_score': 'labels'})
train_dataset = Dataset.from_pandas(train[['input_string', 'labels']].reset_index(drop=True))
val_dataset = Dataset.from_pandas(val[['input_string', 'labels']].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test[['input_string', 'labels']].reset_index(drop=True))

In [20]:
# Tokenize

from transformers import BertTokenizer  # Switch from DistilBertTokenizer

tokenizer = BertTokenizer.from_pretrained('gaunernst/bert-mini-uncased', cache_dir='/kaggle/working/cache')
# tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', cache_dir='/kaggle/working/cache')

tokenizer_config.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/530 [00:00<?, ?B/s]

In [21]:
def tokenize_function(examples):
    return tokenizer(examples['input_string'], truncation=True, padding='max_length', max_length=256)

In [22]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [23]:
print("Train dataset features:", train_dataset.features)
print("First train sample:", train_dataset[0])

Train dataset features: {'input_string': Value(dtype='string', id=None), 'labels': Value(dtype='float64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}
First train sample: {'input_string': "\n    [CONTEXT]\n    Analyze the following user-AI interaction based on the provided text and pre-computed quality metrics.\n    \n    [USER PROMPT]\n    Name the capital of Brazil and the two largest rivers that flow through the country.\n    \n    [AI RESPONSE]\n    Brasília is the capital of Brazil. The Amazon River and the Paraná River are the two largest rivers flowing through the country. The Amazon, the world's largest by discharge volume, traverses the northern region. The Paraná, South America's second-longest river system, flows through or borders the southern parts of the na

### Check data input format

In [25]:
# train_dataset[0]

In [26]:
# Set PyTorch format
# train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'hallucination_score'])
# val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'hallucination_score'])
# test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'hallucination_score'])

In [24]:
# After tokenization (cell 28)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
print("Train dataset after set_format:", train_dataset[0])

Train dataset after set_format: {'labels': tensor(8.), 'input_ids': tensor([  101,  1031,  6123,  1033, 17908,  1996,  2206,  5310,  1011,  9932,
         8290,  2241,  2006,  1996,  3024,  3793,  1998,  3653,  1011, 24806,
         3737, 12046,  2015,  1012,  1031,  5310, 25732,  1033,  2171,  1996,
         3007,  1997,  4380,  1998,  1996,  2048,  2922,  5485,  2008,  4834,
         2083,  1996,  2406,  1012,  1031,  9932,  3433,  1033, 21133,  2401,
         2003,  1996,  3007,  1997,  4380,  1012,  1996,  9733,  2314,  1998,
         1996, 28383,  2314,  2024,  1996,  2048,  2922,  5485,  8577,  2083,
         1996,  2406,  1012,  1996,  9733,  1010,  1996,  2088,  1005,  1055,
         2922,  2011, 11889,  3872,  1010, 20811,  2015,  1996,  2642,  2555,
         1012,  1996, 28383,  1010,  2148,  2637,  1005,  1055,  2117,  1011,
         6493,  2314,  2291,  1010,  6223,  2083,  2030,  6645,  1996,  2670,
         3033,  1997,  1996,  3842,  1012,  1031,  3737, 12046,  2015,  10

In [28]:
!rm -rf train_hallucination val_hallucination test_hallucination

In [25]:
print(train_dataset.features)  # Should show 'labels': FloatField or similar
print(train_dataset[0])  # Should include 'labels' as a tensor or value

{'input_string': Value(dtype='string', id=None), 'labels': Value(dtype='float64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}
{'labels': tensor(8.), 'input_ids': tensor([  101,  1031,  6123,  1033, 17908,  1996,  2206,  5310,  1011,  9932,
         8290,  2241,  2006,  1996,  3024,  3793,  1998,  3653,  1011, 24806,
         3737, 12046,  2015,  1012,  1031,  5310, 25732,  1033,  2171,  1996,
         3007,  1997,  4380,  1998,  1996,  2048,  2922,  5485,  2008,  4834,
         2083,  1996,  2406,  1012,  1031,  9932,  3433,  1033, 21133,  2401,
         2003,  1996,  3007,  1997,  4380,  1012,  1996,  9733,  2314,  1998,
         1996, 28383,  2314,  2024,  1996,  2048,  2922,  5485,  8577,  2083,
         1996,  2406,  1012,  1996,  9733,  1010,  1996,  2088,  1005,  

In [26]:
# Save datasets
train_dataset.save_to_disk('train_hallucination')
val_dataset.save_to_disk('val_hallucination')
test_dataset.save_to_disk('test_hallucination')

print("Preprocessing complete. Datasets saved.")

Saving the dataset (0/1 shards):   0%|          | 0/14 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2 [00:00<?, ? examples/s]

Preprocessing complete. Datasets saved.


## Model training

In [124]:
# import torch
# import torch.nn as nn
# from transformers import BertModel, BertTokenizer  # Switch to Bert* for bert-mini-uncased (explained in section 2)
# from peft import LoraConfig, get_peft_model  # For LoRA

# class HallucinationRegressor(nn.Module):
#     def __init__(self, base_model_name='gaunernst/bert-mini-uncased', use_lora=False, lora_rank=8):
#         super().__init__()
#         try:
#             self.base_model = BertModel.from_pretrained(
#                 base_model_name,
#                 cache_dir='/kaggle/working/cache'
#             )
#             if use_lora:
#                 # Configure LoRA (target attention layers for efficiency)
#                 peft_config = LoraConfig(
#                     r=lora_rank,  # Low-rank dimension (8-32 typical; higher=better quality, more params)
#                     lora_alpha=16,  # Scaling factor
#                     lora_dropout=0.1,
#                     bias="none",
#                     target_modules=["query", "key", "value"]  # Focus on attention for BERT-like models
#                 )
#                 self.base_model = get_peft_model(self.base_model, peft_config)
#                 self.base_model.print_trainable_parameters()  # Logs % trainable params (should be low)
#                 print(f"LoRA enabled with rank {lora_rank}")
#             print(f"Successfully loaded {base_model_name}")
#         except Exception as e:
#             print(f"Failed to load model: {e}")
#             raise
#         # Regression head on pooled output
#         self.regression_head = nn.Linear(self.base_model.config.hidden_size, 1)

#     def forward(self, input_ids, attention_mask):
#         outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
#         pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] token pooling
#         hallucination_score = self.regression_head(pooled_output).squeeze(-1)
#         return {'hallucination_score': hallucination_score}

# # class HallucinationRegressor(nn.Module):
# #     def __init__(self, base_model_name='distilbert-base-uncased'):
# #         super().__init__()
# #         try:
# #             # Load directly with DistilBertModel to bypass AutoModel issues
# #             self.bert = DistilBertModel.from_pretrained(
# #                 base_model_name,
# #                 cache_dir='/kaggle/working/cache'  # Use your cleared working cache
# #             )
# #             print(f"Successfully loaded {base_model_name}")
# #         except Exception as e:
# #             print(f"Failed to load model: {e}")
# #             raise
# #         self.regression_head = nn.Linear(self.bert.config.hidden_size, 1)

# #     def forward(self, input_ids, attention_mask):
# #         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
# #         # Use [CLS] token for pooling (DistilBERT uses last_hidden_state[:, 0, :])
# #         pooled_output = outputs.last_hidden_state[:, 0]
# #         hallucination_score = self.regression_head(pooled_output).squeeze(-1)
# #         return {'hallucination_score': hallucination_score}

In [27]:
# FIXED VERSION - Remove the extra 'self' parameter
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer  # Switch to Bert* for bert-mini-uncased (explained in section 2)
from peft import LoraConfig, get_peft_model  # For LoRA
class HallucinationRegressor(nn.Module):
    def __init__(self, base_model_name='gaunernst/bert-mini-uncased', use_lora=False, lora_rank=8):
        super().__init__()  # Remove the 'self' parameter here
        try:
            self.base_model = BertModel.from_pretrained(
                base_model_name,
                cache_dir='/kaggle/working/cache'
            )
            if use_lora:
                peft_config = LoraConfig(
                    r=lora_rank,
                    lora_alpha=16,
                    lora_dropout=0.1,
                    bias="none",
                    target_modules=["query", "key", "value"]
                )
                self.base_model = get_peft_model(self.base_model, peft_config)
                self.base_model.print_trainable_parameters()
                print(f"LoRA enabled with rank {lora_rank}")
            print(f"Successfully loaded {base_model_name}")
        except Exception as e:
            print(f"Failed to load model: {e}")
            raise
        self.regression_head = nn.Linear(self.base_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        logits = self.regression_head(pooled_output).squeeze(-1)
        
        loss = None
        if labels is not None:
            loss = nn.MSELoss()(logits, labels)
        
        return {"loss": loss, "logits": logits}

# Now this should work:
# model = HallucinationRegressor(use_lora=True)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )

# trainer.train()



In [28]:
import torch
import torch.nn as nn
from transformers import BertModel
from peft import LoraConfig, get_peft_model, TaskType  # Ensure TaskType is imported

class HallucinationRegressor(nn.Module):
    def __init__(self, use_lora=False, lora_rank=8):
        super().__init__()
        self.base_model = BertModel.from_pretrained('gaunernst/bert-mini-uncased')
        self.regression_head = nn.Linear(self.base_model.config.hidden_size, 1)
        
        if use_lora:
            lora_config = LoraConfig(
                task_type=TaskType.FEATURE_EXTRACTION,  # <-- CHANGE HERE: Use FEATURE_EXTRACTION for encoder-only
                inference_mode=False,
                r=lora_rank,
                lora_alpha=32,
                lora_dropout=0.1,
                target_modules=["query", "value", "key"]
            )
            self.base_model = get_peft_model(self.base_model, lora_config)
            self.base_model.print_trainable_parameters()
        
        # Freeze base unless LoRA (already handled by PEFT)
    
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        if token_type_ids is not None:
            del token_type_ids  # Avoid passing unused token_type_ids to base_model
        
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.regression_head(pooled_output)
        
        # Compute loss only if labels provided (for Trainer compatibility)
        loss = None
        if labels is not None:
            loss = nn.MSELoss()(logits, labels)
        
        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

In [29]:
# Load datasets
train_dataset = load_from_disk('train_hallucination')
val_dataset = load_from_disk('val_hallucination')

In [30]:
train_dataset = train_dataset.with_format("torch")
val_dataset = val_dataset.with_format("torch")

In [46]:
print(train_dataset.features)  # Should show 'labels': FloatField or similar
print(train_dataset[0])  # Should include 'labels' as a tensor or value

{'input_string': Value(dtype='string', id=None), 'labels': Value(dtype='float64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}
{'input_string': 'We have Response prompt pairs , where prompt is Explain quantum mechanics simply and\n    the corresponding response is Quantum mechanics studies how particles like electrons behave at tiny scales, often acting like waves and particles simultaneously. where the feat 1 is 0.45\n    give me the hallucination score as output', 'labels': tensor(0.1000), 'input_ids': tensor([  101,  2057,  2031,  3433, 25732,  7689,  1010,  2073, 25732,  2003,
         4863,  8559,  9760,  3432,  1998,  1996,  7978,  3433,  2003,  8559,
         9760,  2913,  2129,  9309,  2066, 15057, 16582,  2012,  4714,  9539,
         1010,  2411,  3772,  2066,  

In [149]:
# Loss function
# def compute_loss(model, inputs, return_outputs=False):
#     labels = inputs.pop('hallucination_score').to(model.device)
#     outputs = model(**inputs)
#     loss = nn.MSELoss()(outputs['hallucination_score'], labels)
#     return (loss, outputs) if return_outputs else loss

In [56]:
# def compute_loss(model, inputs, return_outputs=False):
#     print("Inputs to compute_loss:", inputs.keys())  # Debug
#     labels = inputs.pop('labels').to(model.device)
#     outputs = model(**inputs)
#     loss = nn.MSELoss()(outputs['hallucination_score'], labels)
#     return (loss, outputs) if return_outputs else loss

In [31]:
# Update Cell 34 to use 'logits' (not 'hallucination_score')
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop('labels').to(model.device)
    outputs = model(**inputs)
    loss = nn.MSELoss()(outputs['logits'], labels)  # Fix key to 'logits'
    return (loss, outputs) if return_outputs else loss

In [78]:

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='/kaggle/working/hallucination_results',
    num_train_epochs=48,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    warmup_steps=10,
    gradient_accumulation_steps=1,
    weight_decay=0.01,
    logging_dir='/kaggle/working/hallucination_logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    learning_rate=5e-4,
    fp16=True,
    report_to="none",
)

In [79]:
def custom_data_collator(features):
    """Custom collator that preserves labels"""
    batch = {}
    batch['input_ids'] = torch.stack([f['input_ids'] for f in features])
    batch['attention_mask'] = torch.stack([f['attention_mask'] for f in features])
    batch['labels'] = torch.stack([f['labels'] for f in features])
    return batch

## Custom trainer

In [80]:
# # Custom Trainer
# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         return compute_loss(model, inputs, return_outputs)

In [81]:
# from transformers import Trainer

# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
#         return compute_loss(model, inputs, return_outputs)

In [82]:
# # print("Dataset sample:", train_dataset[0])
# print("Dataset columns after set_format:", list(train_dataset[0].keys()))
# sample_batch = data_collator([train_dataset[0], train_dataset[1]])
# print("Collated batch keys:", list(sample_batch.keys()))

In [83]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer, BertConfig
from peft import LoraConfig, get_peft_model, TaskType

In [84]:
model = HallucinationRegressor(use_lora=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

trainable params: 49,152 || all params: 11,219,712 || trainable%: 0.4381


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,82.789543
2,No log,81.451683
3,No log,78.930336
4,No log,75.018799
5,61.667200,69.354630
6,61.667200,62.138542
7,61.667200,55.649681
8,61.667200,51.059120
9,61.667200,47.771805
10,45.530500,44.947052


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, targe

TrainOutput(global_step=96, training_loss=23.480180025100708, metrics={'train_runtime': 11.2079, 'train_samples_per_second': 59.958, 'train_steps_per_second': 8.565, 'total_flos': 0.0, 'train_loss': 23.480180025100708, 'epoch': 48.0})

In [39]:
# trainer = CustomTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=custom_data_collator,  # Use the custom collator
# )

# # Step 7: Test before training
# try:
#     # Test that batching works
#     sample_batch = next(iter(trainer.get_train_dataloader()))
#     print("✓ Batch creation successful")
#     print("Batch contains:", list(sample_batch.keys()))
#     print("Labels shape:", sample_batch['labels'].shape)
    
#     # Start training
#     trainer.train()
#     trainer.save_model('fine_tuned_hallucination_lora')
#     print("✓ Training completed successfully!")
    
# except Exception as e:
#     print(f"✗ Error: {e}")
#     # Additional debugging
#     print("Dataset features:", train_dataset.features)
#     print("First sample:", {k: type(v) for k, v in train_dataset[0].items()})

In [40]:
# from transformers import DefaultDataCollator

# from transformers import DataCollatorWithPadding
# from torch.utils.data import DataLoader

# try:
#     # data_collator = DefaultDataCollator(return_tensors="pt")
#     model = HallucinationRegressor(use_lora=True)
#     trainer = CustomTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=val_dataset,
#         data_collator=custom_data_collator,  # Use the custom collator
#         )
#     # In training cell, before trainer.train()
#     print("First batch sample:", next(iter(trainer.get_train_dataloader()))['labels'])
#     trainer.train()
#     trainer.save_model('fine_tuned_hallucination_lora')
#     print("Training complete with LoRA. Model saved.")
# except Exception as e:
#     print(f"Training failed: {e}")
#     raise

In [41]:
# # Train (enable LoRA here)
# try:
#     model = HallucinationRegressor(use_lora=True)  # Add use_lora=True
#     trainer = CustomTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=val_dataset,
#     )
#     trainer.train()
#     trainer.save_model('fine_tuned_hallucination_lora')
#     print("Training complete with LoRA. Model saved.")
# except Exception as e:
#     print(f"Training failed: {e}")
#     raise

# # # Train
# # try:
# #     model = HallucinationRegressor()
# #     trainer = CustomTrainer(
# #         model=model,
# #         args=training_args,
# #         train_dataset=train_dataset,
# #         eval_dataset=val_dataset,
# #     )
# #     trainer.train()
# # except Exception as e:
# #     print(f"Training failed: {e}")
# #     raise
    

In [42]:
# # Save model
# trainer.save_model('fine_tuned_hallucination')
# print("Training complete. Model saved.")

Training complete. Model saved.


In [40]:
# Save model
# import torch
# import os
# from transformers import BertTokenizer

# save_path = '/kaggle/working/my_model'
# os.makedirs(save_path, exist_ok=True)

# torch.save(trainer.model.state_dict(), f'{save_path}/weights.pt')
# tokenizer = BertTokenizer.from_pretrained('gaunernst/bert-mini-uncased')
# tokenizer.save_pretrained(f'{save_path}/tokenizer')

# print("Model saved")

tokenizer_config.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/530 [00:00<?, ?B/s]

Model saved


In [85]:
import os
import torch
from transformers import BertConfig

# Define save path
save_path = '/kaggle/working/fine_tuned_model'
os.makedirs(save_path, exist_ok=True)

# 1. Save full model weights as pytorch_model.bin (includes base, LoRA, regression head)
torch.save(trainer.model.state_dict(), os.path.join(save_path, 'pytorch_model.bin'))
print("✓ Saved pytorch_model.bin")

# 2. Save base config as config.json
config = BertConfig.from_pretrained('gaunernst/bert-mini-uncased')
config.num_labels = 1  # For regression head
config.save_pretrained(save_path)
print("✓ Saved config.json")

# 3. Save tokenizer (generates tokenizer_config.json, special_tokens_map.json, vocab.txt)
tokenizer.save_pretrained(save_path)
print("✓ Saved tokenizer files")

print(f"\nModel saved to {save_path}/")
print("Files created:")
for file in sorted(os.listdir(save_path)):
    print(f"  - {file}")

✓ Saved pytorch_model.bin
✓ Saved config.json
✓ Saved tokenizer files

Model saved to /kaggle/working/fine_tuned_model/
Files created:
  - config.json
  - pytorch_model.bin
  - special_tokens_map.json
  - tokenizer_config.json
  - vocab.txt


## Predict

In [55]:
import torch
from transformers import BertTokenizer

save_path = '/kaggle/working/my_model'
tokenizer = BertTokenizer.from_pretrained(f'{save_path}/tokenizer')
model = HallucinationRegressor(use_lora=True)
model.load_state_dict(torch.load(f'{save_path}/weights.pt', map_location='cpu', weights_only=False))
model.eval()

trainable params: 49,152 || all params: 11,219,712 || trainable%: 0.4381
LoRA enabled with rank 8
Successfully loaded gaunernst/bert-mini-uncased


HallucinationRegressor(
  (base_model): PeftModel(
    (base_model): LoraModel(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 256, padding_idx=0)
          (position_embeddings): Embedding(512, 256)
          (token_type_embeddings): Embedding(2, 256)
          (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-3): 4 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=256, out_features=256, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=256, o

In [74]:
def load_hallucination_model(save_path='/kaggle/working/fine_tuned_model'):
    if not os.path.exists(save_path):
        raise ValueError(f"Model directory not found: {save_path}")
    
    # Load config
    config = BertConfig.from_pretrained(save_path)
    
    # Initialize model with LoRA (since it was trained with LoRA)
    model = HallucinationRegressor(use_lora=True)
    
    # Load weights
    state_dict = torch.load(os.path.join(save_path, 'pytorch_model.bin'), map_location='cpu')
    model.load_state_dict(state_dict)
    
    # Set to eval mode
    model.eval()
    
    # Load tokenizer
    tokenizer = BertTokenizer.from_pretrained(save_path)
    
    print("Model and tokenizer loaded successfully!")
    return model, tokenizer

# Example usage
model, tokenizer = load_hallucination_model()

trainable params: 49,152 || all params: 11,219,712 || trainable%: 0.4381
Model and tokenizer loaded successfully!


In [75]:
def predict_hallucination(model, tokenizer, prompt, response, jaccard_score):
    input_text = f"""We have Response prompt pairs , where prompt is {prompt} and
    the corresponding response is {response} where the feat 1 is {jaccard_score}
    give me the hallucination score as output"""
    
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=256
    )
    
    # Remove token_type_ids if your model doesn't need them (BERT-mini might)
    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']
    
    with torch.no_grad():
        outputs = model(**inputs)
        hallucination_score = outputs['logits'].item()
    
    return max(0, min(1, hallucination_score))

In [76]:
prompt = "What is the capital of France?"
response = "The capital is Paris, known for its Eiffel Tower."
jaccard = 0.75
score = predict_hallucination(model, tokenizer, prompt, response, jaccard)
print(f"Hallucination Score: {score:.3f}")

Hallucination Score: 0.220


In [56]:
# def predict_hallucination(prompt, response, jaccard):
#     data = pd.DataFrame({'prompt': [prompt], 'response': [response], 'jaccard': [jaccard]})
#     input_string = bert_input_string(0, data)  # Use the function above
#     inputs = tokenizer(
#         input_string, 
#         return_tensors='pt', 
#         truncation=True, 
#         padding='max_length', 
#         max_length=256
#     )
#     with torch.no_grad():
#         outputs = model(**inputs)
#     score = outputs['hallucination_score'].item()
#     return score

In [57]:
def predict_hallucination(model, tokenizer, prompt, response, jaccard_score):
    input_text = f"""We have Response prompt pairs , where prompt is {prompt} and
    the corresponding response is {response} where the feat 1 is {jaccard_score}
    give me the hallucination score as output"""
    
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=256
    )
    
    # Remove token_type_ids if present
    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']
    
    with torch.no_grad():
        outputs = model(**inputs)
        hallucination_score = outputs['logits'].item()
    
    return max(0, min(1, hallucination_score))

In [58]:
# # Example usage
# examples = [
#     ("What is the capital of France?", "The capital is Paris, known for its Eiffel Tower.", 0.75),
#     ("Who won the 2024 Super Bowl?", "The Kansas City Chiefs won, beating the Philadelphia Eagles.", 0.60),
#     ("Explain dark matter", "Dark matter is invisible stuff that makes up most of the universe and pulls galaxies together with gravity.", 0.50)
# ]

# for prompt, response, jaccard in examples:
#     score = predict_hallucination(prompt, response, jaccard)
#     print(f"Prompt: {prompt}\nResponse: {response}\nJaccard: {jaccard}\nHallucination Score: {score:.3f} (0=low, 1=high)\n---")

TypeError: predict_hallucination() missing 2 required positional arguments: 'response' and 'jaccard_score'

In [ ]:

# Then use the correct function signature
examples = [
    ("What is the capital of France?", "The capital is delhi, known for its taj mahal.", 0.15),
    ("Who won the 2024 Super Bowl?", "The Kansas City Chiefs won, beating the Philadelphia Eagles.", 0.60),
    ("Explain dark matter", "Dark matter is invisible stuff that makes up most of the universe and pulls galaxies together with gravity.", 0.30)
]

if model is not None and tokenizer is not None:
    for prompt, response, jaccard in examples:
        score = predict_hallucination(model, tokenizer, prompt, response, jaccard)  # Add model and tokenizer
        print(f"Prompt: {prompt}\nResponse: {response}\nJaccard: {jaccard}\nHallucination Score: {score:.3f} (0=low, 1=high)\n---")
else:
    print("Model or tokenizer failed to load. Check your training completed successfully.")